In [1]:
# initialize findspark for local
import findspark
findspark.init('/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7')

In [2]:
import configparser
from datetime import datetime
# import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
# Configure the necessary Spark environment
import os
import sys
import boto3

In [4]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.1 pyspark-shell'

In [5]:
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, "/Users/johnrick/opt/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"))

# Initialize PySpark
exec(open(os.path.join(spark_home, "/Users/johnrick/Downloads/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py")).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
SparkSession available as 'spark'.


In [6]:
from pyspark.sql.types import StructType as R, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType, StructType, LongType

In [7]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [8]:
song_schema = StructType([
        StructField("artist_id", StringType(), False),
        StructField("artist_latitude", StringType(), True),
        StructField("artist_longitude", StringType(), True),
        StructField("artist_location", StringType(), True),
        StructField("artist_name", StringType(), False),
        StructField("song_id", StringType(), False),
        StructField("title", StringType(), False),
        StructField("duration", DoubleType(), False),
        StructField("year", IntegerType(), False)
])

In [9]:
ls

README.md         dl.cfg            get_data.ipynb    ud-s3datalake/
data/             etl.py            test_local.ipynb  ud-s3datalake-ap/


In [14]:
song_data = "data/song_data/*/*/*/*.json"

In [15]:
df = spark.read.json(song_data, schema=song_schema)